Your Own JARVIS

In [18]:
import openai
from openai import OpenAI
import whisper
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import pyaudio
import wave
import os

In [19]:
# Your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

In [20]:
# Initialize the Whisper model
model = whisper.load_model("base")

In [21]:
# Function to record audio from the microphone
def record_audio(filename, duration, fs=44100, channels=1):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=fs,
                    input=True,
                    frames_per_buffer=1024)
    
    print("Recording...")
    frames = []

    for _ in range(0, int(fs / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    
    print("Recording finished")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wave_file = wave.open(filename, 'wb')
    wave_file.setnchannels(channels)
    wave_file.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wave_file.setframerate(fs)
    wave_file.writeframes(b''.join(frames))
    wave_file.close()

In [22]:
# Path to your audio file
recorded_audio_path = "D:/Generative-AI/GenAI-Demo-Projects/02-Your-JARVIS/recorded_audio.wav"

# Path to save the response audio
response_audio_path = "D:/Generative-AI/GenAI-Demo-Projects/02-Your-JARVIS/response.mp3"

In [23]:
# Function to convert speech to text using Whisper
def speech_to_text(audio_path):
    result = model.transcribe(recorded_audio_path)
    return result["text"]

In [24]:
# Function to get response from GPT-
def get_gpt4_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant, respond in short to the user questions."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150
    )
    #return response['choices'][0]['message']['content'].strip()
    return response.choices[0].message.content.strip()

In [25]:
# Function to convert text to speech
def text_to_speech(text, output_path):
    tts = gTTS(text)
    tts.save(output_path)
    sound = AudioSegment.from_file(output_path)
    play(sound)

In [26]:
# Record audio from the microphone
record_duration = 9  # seconds
record_audio(recorded_audio_path, record_duration)


Recording...
Recording finished


In [27]:
# Convert speech to text
print("Converting speech to text...")
prompt = speech_to_text(recorded_audio_path)
print(f"Recognized text: {prompt}")



Converting speech to text...
Recognized text:  Hello, I am Harshat Bavar, can you introduce yourself?


In [29]:
# Get response from GPT-4
print("Getting response from GPT-4...")
response = get_gpt4_response(prompt, api_key)
print(f"GPT-4 response: {response}")

Converting response text to speech...
Done!


In [ ]:
# Convert response text to speech
print("Converting response text to speech...")
text_to_speech(response, response_audio_path)
print("Done!")